# cTraderFix Jupyter Sample

In this Jupyter notebook we will use the Python package "ctrader-fix" to interact with cTrader FIX API.

Let's start.

If you haven't already installed the "ctrader-fix" package, run the next code cell to install it via pip:

In [1]:
!pip install ctrader-open-api

^C


Then we have to import all necessary types:

In [2]:
from twisted.internet import reactor
import json
from ctrader_fix import *
import datetime

Now we use the "config-trade.json" config file to get your API credentials.
Be sure to populate it with your API credentials before running next cell:

In [3]:
# you can use two separate config files for QUOTE and TRADE
with open("config.json") as configFile:
    config = json.load(configFile)

Then we will create a client based on our config:

In [4]:
client = Client(config["Host"], config["Port"], ssl = config["SSL"])

We will use ther below "send" method for sending our message:

In [5]:
def send(request):
    diferred = client.send(request)
    diferred.addCallback(lambda _: print("\nSent: ", request.getMessage(client.getMessageSequenceNumber()).replace("", "|")))

We are going to:

1. Send a log on requst
2. After receiving log on response we send security list request
3. Then if host is Trade we create a new market order for second symbol otherwise we send a mrket data request for the second symbol

Let's set the client call backs that will be used by client after it got message received/connected/disconnected:

In [6]:
def onMessageReceived(client, responseMessage): # Callback for receiving all messages
    print("Received: ", responseMessage.getMessage().replace("", "|"))
    # We get the message type field value
    messageType = responseMessage.getFieldValue(35)
    # we send a security list request after we received logon message response
    if messageType == "A":
        securityListRequest = SecurityListRequest(config)
        securityListRequest.SecurityReqID = "A"
        securityListRequest.SecurityListRequestType = 0
        send(securityListRequest)
    # After receiving the security list we send a market order request by using the security list first symbol
    elif messageType == "y":
        # We use getFieldValue to get all symbol IDs, it will return a list in this case
        # because the symbol ID field is repetitive
        symboldIds = responseMessage.getFieldValue(55)
        if config["TargetSubID"] == "TRADE":
            newOrderSingle = NewOrderSingle(config)
            newOrderSingle.ClOrdID = "B"
            newOrderSingle.Symbol = symboldIds[1]
            newOrderSingle.Side = 1
            newOrderSingle.OrderQty = 1000
            newOrderSingle.OrdType = 1
            newOrderSingle.Designation = "From Jupyter"
            send(newOrderSingle)
        else:
            marketDataRequest = MarketDataRequest(config)
            marketDataRequest.MDReqID = "a"
            marketDataRequest.SubscriptionRequestType = 1
            marketDataRequest.MarketDepth = 1
            marketDataRequest.NoMDEntryTypes = 1
            marketDataRequest.MDEntryType = 0
            marketDataRequest.NoRelatedSym = 1
            marketDataRequest.Symbol = symboldIds[1]
            send(marketDataRequest)
    # after receiving the new order request response we stop the reactor
    # And we will be disconnected from API
    elif messageType == "8" or messageType == "j":
        print("We are done, stopping the reactor")
        reactor.stop()

def disconnected(client, reason): # Callback for client disconnection
    print("Disconnected, reason: ", reason)

def connected(client): # Callback for client connection
    print("Connected")
    logonRequest = LogonRequest(config)
    send(logonRequest)
    
# Setting client callbacks
client.setConnectedCallback(connected)
client.setDisconnectedCallback(disconnected)
client.setMessageReceivedCallback(onMessageReceived)

The last step is to run our client service, it will run inside Twisted reactor loop asynchronously:

In [ ]:
# Starting the client service
client.startService()
# Run Twisted reactor, we imported it earlier
reactor.run()

  Using cached twisted-24.3.0-py3-none-any.whl.metadata (9.5 kB)
Using cached twisted-24.3.0-py3-none-any.whl (3.2 MB)
  Attempting uninstall: Twisted
    Found existing installation: Twisted 21.7.0
    Uninstalling Twisted-21.7.0:
      Successfully uninstalled Twisted-21.7.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ctrader-fix 0.1.1 requires Twisted==21.7.0, but you have twisted 24.3.0 which is incompatible.
